In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
import ktrain
ktrain.__version__

I0907 08:21:33.460679 140294714414912 file_utils.py:39] PyTorch version 1.4.0 available.
I0907 08:21:33.464329 140294714414912 file_utils.py:55] TensorFlow version 2.1.0 available.


'0.21.3'

# Learning from Unlabeled Text Data

Unlabeled, unstructured text or document data abound, and it is often necessary to "make sense" of these data for various applications.  Examples include:
- *exploratory analysis of text data*:  provide rich overviews of the information space to discover relevant information for which one may not have even known to look
- *building training sets for text classification*:  identifying positive and negative example documents to train a [text classifier](https://en.wikipedia.org/wiki/Document_classification) in a semi-automated fashion 
- *document similarity*: measuring the semantic simlarity between documents or sets of documents
- *document recommender systems*:  given a specific document of interest, recommend other documents that are semantically similar to it

Each of these examples involve **learning from largely unlabeled text data**.  In this notebook, we will show you how to accomplish the above with minimal coding using *ktrain*.    The *ktrain* library is an open-source, augmented ML library built around Keras and scikit-learn.  It can be installed with `pip3 install ktrain` and is [available on GitHub](https://github.com/amaiya/ktrain).

We will use the well-known [20-newsgroup dataset](http://qwone.com/~jason/20Newsgroups/) for this demonstration.

## Get Raw Document Data

In [3]:
# 20newsgroups
from sklearn.datasets import fetch_20newsgroups

# we only want to keep the body of the documents!
remove = ('headers', 'footers', 'quotes')

# fetch train and test data
newsgroups_train = fetch_20newsgroups(subset='train', remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', remove=remove)

# compile the texts
texts = newsgroups_train.data +  newsgroups_test.data

# let's also store the newsgroup category associated with each document
# we can display this information in visualizations
targets = [target for target in list(newsgroups_train.target) + list(newsgroups_test.target)]
categories = [newsgroups_train.target_names[target] for target in targets]

We are loading the targets (i.e., newsgroup categories), but will not use them for learning a model.  Rather, they are simply employed as an example of how to incorporate metadata about documents in visualizations and anlayses.

## Train an LDA Topic Model to Discover Topics

The `get_topic_model` function learns a [topic model](https://en.wikipedia.org/wiki/Topic_model) using [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

The `n_features` argument specifies the size of the vocabulary, and the `n_topics` argument sets the number of topics (or clusters) to discover. 

In [4]:
%%time
tm = ktrain.text.get_topic_model(texts, n_topics=None, n_features=10000)

n_topics automatically set to 97
lang: en
preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.
CPU times: user 15min 40s, sys: 42min 16s, total: 57min 57s
Wall time: 1min 56s


 We can examine the discovered topics using `print_topics`, `get_topics`, or `topics`.  Here, we will use `print_topics`:

In [5]:
tm.print_topics()

topic 0 | tape adam tim case moved bag quote mass marked zionism
topic 1 | image jpeg images format programs tiff files jfif save lossless
topic 2 | alternative movie film static cycles films philips dynamic hou phi
topic 3 | hell humans poster frank reality kent gerard gant eternal bell
topic 4 | air phd chz kit cbc ups w-s rus w47 mot
topic 5 | dog math great figure poster couldn don trying rushdie fatwa
topic 6 | collaboration nazi fact end expression germany philly world certified moore
topic 7 | gif points scale postscript mirror plane rendering algorithm polygon rayshade
topic 8 | fonts font shell converted iii characters slight composite breaks compress
topic 9 | power station supply options option led light tank plastic wall
topic 10 | transmission rider bmw driver automatic shift gear japanese stick highway
topic 11 | tyre ezekiel ruler hernia appeared appointed supreme man land power
topic 12 | space nasa earth data launch surface solar moon mission planet
topic 13 | israel j

From the above, we can immediately get a feel for what kinds of subjects are discussed within this dataset.  For instsance, Topic \#13 appears to be about the Middle East with labels: "*israel jews jewish israeli arab peace*".

We can examine the word weights for this topic, where the "weight" is a pseudo-count (that can be converted to a probability if normalizing over all words in vocabulary):

In [6]:
tm.get_word_weights(topic_id=13, n_words=25)

[('israel', 832.6639019298735),
 ('jews', 670.7851381220836),
 ('jewish', 539.6844212023212),
 ('israeli', 426.2434455755054),
 ('arab', 376.1511464659808),
 ('peace', 269.33000902133085),
 ('war', 229.3267450288203),
 ('arabs', 223.59854716988627),
 ('palestinian', 191.29031007300767),
 ('kuwait', 182.83909796210693),
 ('land', 173.23994664154932),
 ('palestinians', 158.31898772111572),
 ('state', 151.5461024601567),
 ('palestine', 121.14271427446522),
 ('west', 113.54334001125812),
 ('iraq', 111.54753922935043),
 ('jew', 106.56410679110718),
 ('attacks', 106.47122225017874),
 ('israelis', 99.7315473662208),
 ('gaza', 98.13080829032971),
 ('killed', 92.8557266909479),
 ('occupied', 90.79070215809577),
 ('country', 89.68676723731086),
 ('policy', 86.85153167299889),
 ('civilians', 86.63190206561713)]

## Computing the Document-Topic Matrix

We will now pre-compute the document-topic matrix.  Each row in this matrix represents a document, and the columns represent the probability distribution over the 97 topics.  This allows us to easily see what kinds of topics are covered by any specific document in the original corpus.

When computing the document-topic matrix, we will also filter out documents whose maximum topic probability is less than 0.25 in order to consider the most representative documents for each topic.  This may help to improve clarity of visualizations (shown later) by removing "unfocused" documents. 

In [7]:
%%time
tm.build(texts, threshold=0.25)

done.
CPU times: user 1min 21s, sys: 3min 20s, total: 4min 42s
Wall time: 12.1 s


Since the `build` method prunes documents based on threshold, we should prune the original data and any metadata in a similar way for consistency.  This can be accomplished with the `filter` method. 

In [8]:
texts = tm.filter(texts)
categories = tm.filter(categories)

This is useful to ensure all data and metadata are aligned with the same array indices in case we want to use them later (e.g., in visualizations, for example).

Having computed the document-topic matrix, we can now easily access the topic probablity distribution for any document in the corpus using `get_doctopics`.   For instance, this document in the corpus is about sports:

In [9]:
print(texts[35])

For the second straight game, California scored a ton of late runs to crush
the Brewhas. It was six runs in the 8th for a 12-5 win Monday and five in
the 8th and six in the 9th for a 12-2 win yesterday. Jamie Navarro pitched
seven strong innings, but Orosco, Austin, Manzanillo and Lloyd all took part
in the mockery of a bullpen yesterday. How's this for numbers? Maldanado has
pitched three scoreless innings and Navarro's ERA is 0.75. The next lowest
on the staff is Wegman at 5.14. Ouch!

It doesn't look much better for the hitters. Hamilton is batting .481, while
Thon is hitting .458 and has seven RBI. The next highest is three. The next
best hitter is Jaha at .267 and then Vaughn, who has the team's only HR, at
.238. Another ouch. Looking at the stats, it's not hard to see why the team
is 2-5. In fact, 2-5 doesn't sound bad when you're averaging three runs/game
and giving up 6.6/game. 

Still, it's early and things will undoubtedly get better. The offense should
come around, but the b

And, here is the topic probability distribution for this document:

In [10]:
tm.get_doctopics(doc_ids=[35])

array([[4.64381931e-04, 4.64381908e-04, 4.64381908e-04, 4.64381909e-04,
        4.64381908e-04, 4.64381908e-04, 4.64381908e-04, 4.64381908e-04,
        4.64381908e-04, 4.64381911e-04, 4.64381908e-04, 4.64381908e-04,
        4.64381912e-04, 4.64381914e-04, 4.64381908e-04, 4.64381909e-04,
        4.64381908e-04, 4.64381909e-04, 4.64381908e-04, 4.64381922e-04,
        4.64381909e-04, 8.60305392e-02, 4.64381913e-04, 4.64381908e-04,
        4.64381908e-04, 4.64381908e-04, 4.64381908e-04, 4.64381915e-04,
        4.64381908e-04, 4.64381924e-04, 4.64381908e-04, 4.64381919e-04,
        4.64381947e-04, 4.64381915e-04, 4.64381908e-04, 4.64381908e-04,
        4.64381908e-04, 4.64381908e-04, 4.64381908e-04, 4.64381908e-04,
        4.64381908e-04, 4.64381908e-04, 4.64381926e-04, 4.64381929e-04,
        4.64381908e-04, 4.64381911e-04, 6.93731322e-01, 4.64381908e-04,
        4.64381908e-04, 4.64381908e-04, 1.50902793e-02, 4.64381911e-04,
        4.64381909e-04, 9.05690040e-03, 4.64381908e-04, 4.643819

As expected, the highest topic probability (69%) is associated with a topic about sports:

In [11]:
tm.topics[ np.argmax(tm.get_doctopics(doc_ids=[35]))]

'game team games year hockey season players player baseball league'

## Predicting the Topics of New Documents

The `predict` method can predict the topic probability distribution for any arbitrary document directly from raw text:

In [12]:
tm.predict(['Elon Musk leads Space Exploration Technologies (SpaceX), where he oversees '  +
            'the development and manufacturing of advanced rockets and spacecraft for missions ' +
            'to and beyond Earth orbit.'])

array([[0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.65009096, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.06185567, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00303214, 0.00303214,
        0.00303214, 0.00303214, 0.00303214, 0.00

As expected, the highest topic probability for this sentence is from topic \#12 (third row and third column), which is about space and related things:

In [13]:
tm.topics[ np.argmax(tm.predict(['Elon Musk leads Space Exploration Technologies (SpaceX), where he oversees '  +
            'the development and manufacturing of advanced rockets and spacecraft for missions ' +
            'to and beyond Earth orbit.']))]

'space nasa earth data launch surface solar moon mission planet'

## Visualizing Topics
Let's take another look at the list of discovered topics but sorted by document count.

In [14]:
tm.print_topics(show_counts=True)

topic:79 | count:3782 | like know does use don just good thanks need want
topic:96 | count:3643 | just don think know like time did going didn people
topic:43 | count:1599 | god people does say believe bible true think evidence religion
topic:42 | count:1246 | people government right think rights law make public fbi don
topic:51 | count:900 | card memory windows board ram bus drivers driver cpu problem
topic:46 | count:782 | game team games year hockey season players player baseball league
topic:92 | count:597 | files file edu ftp available version server data use sun
topic:29 | count:399 | edu university information send new computer research mail internet address
topic:82 | count:371 | price new sale offer sell condition shipping interested asking prices
topic:84 | count:312 | armenian armenians people turkish war said killed children russian turkey
topic:12 | count:296 | space nasa earth data launch surface solar moon mission planet
topic:22 | count:283 | key encryption chip keys cl

The topic with the most documents appears to be conversational questions, replies, and comments that aren't focused on a particular subject.  Other topics are focused on specific domains (e.g., topic 15 is about **medicine** with label "*medical health disease cancer patients drug treatment*").


We can easily generate an interactive visualization of the documents under consideration using `visualize_documents`:

In [15]:
tm.visualize_documents(doc_topics=tm.get_doctopics())

reducing to 2 dimensions...[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 15644 samples in 0.173s...
[t-SNE] Computed neighbors for 15644 samples in 35.209s...
[t-SNE] Computed conditional probabilities for sample 1000 / 15644
[t-SNE] Computed conditional probabilities for sample 2000 / 15644
[t-SNE] Computed conditional probabilities for sample 3000 / 15644
[t-SNE] Computed conditional probabilities for sample 4000 / 15644
[t-SNE] Computed conditional probabilities for sample 5000 / 15644
[t-SNE] Computed conditional probabilities for sample 6000 / 15644
[t-SNE] Computed conditional probabilities for sample 7000 / 15644
[t-SNE] Computed conditional probabilities for sample 8000 / 15644
[t-SNE] Computed conditional probabilities for sample 9000 / 15644
[t-SNE] Computed conditional probabilities for sample 10000 / 15644
[t-SNE] Computed conditional probabilities for sample 11000 / 15644
[t-SNE] Computed conditional probabilities for sample 12000 / 15644
[t-SNE] Computed condi

Loading BokehJS ...

## Inspecting Topics

The `get_docs` method allows you to retrieve document data by `doc_id` or `topic_id`. When `rank=True`, documents are sorted based on relevance to the topic.  This is particularly useful for inspecting the most relevant documents to each topic.

Consider Topic \# 51, which appears to be about computer hardware subjects:

In [16]:
tm.topics[51]

'card memory windows board ram bus drivers driver cpu problem'

Let's examine this the top most relevant document to this topic:

In [17]:
doc = tm.get_docs(topic_ids=[51], rank=True)[0]
print('DOC_ID: %s'  % (doc[1]))
print('TOPIC SCORE: %s '% (doc[2]))
print('TOPIC_ID: %s' % (doc[3]))
print('TEXT: %s' % (doc[0]))

DOC_ID: 8252
TOPIC SCORE: 0.7938141074892827 
TOPIC_ID: 51
TEXT: Hi there,

With a 16Megs of RAM, is there a need to run/load Smartdrv for
Windows 3.1?  If yes, can I run/load Ramdrive without Smartdrv?
If I need both Ramdrive & Smartdrv, is the following Config.Sys
settings OK:  ...SMARTDRV.SYS 2048 2048
              ...RAMDRIVE.SYS 2048 /E

Thanks in advance for e-mail reply.


Looks right to me.  Note that the `get_docs` method returns a list of tuples of the form:
`(text, doc_id, probability_score, topic_id)`

... where 
- `text` is the raw text of the document
- `doc_id` is an index into the array returned by `get_doctopics`
- `topic_id` is the index of the topic in the range of `range(n_topics)`
- `probability_score` is the relevance of this document to the topic represented by `topic_id`

When `rank=True`, within each `topic_id`, the tuples are sorted in desceding order by `probability_score`.  Hence, the first item is the most relevant document to the selected topic (`topic_id=51`).  If `rank=False`, results are sorted in ascending order by `doc_id` (i.e,. the same order as `texts` that was supplied as input to `build`.

Next, let's examine some additional topics that appears related to the larger theme of technology.

Here is a topic which appears to be about Windows software:

In [18]:
tm.topics[85]

'dos windows software comp library os/2 version microsoft applications code'

In [19]:
tm.get_docs(topic_ids=[85], rank=True)[0]

("1.  Software publishing SuperBase 4 windows v.1.3           --->$80\n\n2.  OCR System ReadRight v.3.1 for Windows                  --->$65\n\n3.  OCR System ReadRight  v.2.01 for DOS                    --->$65\n\n4.  Unregistered Zortech 32 bit C++ Compiler v.3.1          --->$ 250\n     with Multiscope windows Debugger,\n     WhiteWater Resource Toolkit, Library Source Code\n\n5.  Glockenspiel/ImageSoft Commonview 2 Windows\n     Applications Framework for Borland C++                 --->$70\n\n6.  Spontaneous Assembly Library With Source Code           --->$50\n\n7.  Microsoft Macro Assembly 6.0                            --->$50\n\n8.  Microsoft Windows v.3.1 SDK Documentation               --->$125\n\n9.  Microsoft FoxPro V.2.0                                  --->$75\n\n10.  WordPerfect 5.0 Developer's Toolkit                    --->$20\n\n11.  Kedwell Software DataBoss v.3.5 C Code Generator       --->$100\n\n12.  Kedwell InstallBoss v.2.0 Installation Generator       --->$35\n

A topic about **Programming**:

In [20]:
tm.topics[94]

'file program error output use section line code command problem'

In [21]:
tm.get_docs(topic_ids=[94], rank=True)[0]

('I am trying to write an image display program that uses\nthe MIT shared memory extension.  The shared memory segment\ngets allocated and attached to the process with no problem.\nBut the program crashes at the first call to XShmPutImage,\nwith the following message:\n\nX Error of failed request:  BadShmSeg (invalid shared segment parameter)\n  Major opcode of failed request:  133 (MIT-SHM)\n  Minor opcode of failed request:  3 (X_ShmPutImage)\n  Segment id in failed request 0x0\n  Serial number of failed request:  741\n  Current serial number in output stream:  742\n\nLike I said, I did error checking on all the calls to shmget\nand shmat that are necessary to create the shared memory\nsegment, as well as checking XShmAttach.  There are no\nproblems.\n\nIf anybody has had the same problem or has used MIT-SHM without\nhaving the same problem, please let me know.\n\nBy the way, I am running OpenWindows 3.0 on a Sun Sparc2.',
 42,
 0.8379850047418861,
 94)

A topic about **cryptography**:

In [22]:
tm.topics[22]

'key encryption chip keys clipper phone security use government privacy'

In [23]:
tm.get_docs(topic_ids=[22], rank=True)[0]

('Here is a revised version of my summary which corrects some errors\nand provides some additional information and explanation.\n\n\n                     THE CLIPPER CHIP: A TECHNICAL SUMMARY\n\n                               Dorothy Denning\n\n                           Revised, April 21, 1993\n\n\nINTRODUCTION\n\nOn April 16, the President announced a new initiative that will bring\ntogether the Federal Government and industry in a voluntary program\nto provide secure communications while meeting the legitimate needs of\nlaw enforcement.  At the heart of the plan is a new tamper-proof encryption\nchip called the "Clipper Chip" together with a split-key approach to\nescrowing keys.  Two escrow agencies are used, and the key parts from\nboth are needed to reconstruct a key.\n\n\nCHIP CONTENTS\n\nThe Clipper Chip contains a classified single-key 64-bit block\nencryption algorithm called "Skipjack."  The algorithm uses 80 bit keys\n(compared with 56 for the DES) and has 32 rounds of scra

## Compiling a Sample of Interesting Documents

Let's combine these technology-related documents into a set of positive examples of technology-focused posts.  We can use these documents as seeds to find new documents about technology. To measure semantic similarity among documents, we will represent each document by its topic probability distribution.  

In [24]:
tech_topics = [51, 85, 94, 22]
tech_probs = tm.get_doctopics(topic_ids=tech_topics)
doc_ids = [doc[1] for doc in tm.get_docs(topic_ids=tech_topics)]

Let's visualize these technology-focused documents.  We will also compile the original newsgroup categories for each document, so that they can be included in the visualization. (This is why we invoked the `filter` method earler.)

In [25]:
newsgroup_categories = [categories[doc_id] for doc_id in doc_ids]
tm.visualize_documents(doc_topics=tech_probs, extra_info={'cat': newsgroup_categories, 'doc_id':doc_ids})

reducing to 2 dimensions...[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1393 samples in 0.009s...
[t-SNE] Computed neighbors for 1393 samples in 0.261s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1393
[t-SNE] Computed conditional probabilities for sample 1393 / 1393
[t-SNE] Mean sigma: 0.082664
[t-SNE] KL divergence after 250 iterations with early exaggeration: 65.742264
[t-SNE] KL divergence after 1000 iterations: 0.995215
done.


Loading BokehJS ...

## Scoring Documents by Similarity

Once you've identified a set of documents that are interesting to your use case, you may want to identify additional documents that are semantically similar to this set.  Here, suppose we wanted to identify new documents that are related to computer technology.  We can accomplish this with the `train_scorer` and `score` methods.  The `train_scorer` method compiles a list of seed documents based on supplied `topic_ids` or `doc_ids`.   The `score` method scores new documents based on their similarity to the seed documents. Internally, this is accomplished by training a rudimentary [One-Class classifier](https://en.wikipedia.org/wiki/One-class_classification).  While this classifier can be used as is, it is also useful to use the `score` method to help compile a training set for a traditinal binary classifier.

In [26]:
tm.train_scorer(topic_ids=tech_topics)

We can now invoke the `score` method to measure the degree to which new documents are similar to our technology-related topics.  Note that, although we are applying the scorer to documents within the set corpus used to train the topic model, this is not required. Our `scorer` can be applied to any arbitrary set of documents.

Let's retrieve the text associated with all documents **not** associated with our selected technology-focused topics. These documents stored in `other_texts` are present in the original corpus.  You could also construct `other_texts` to contain an entirely new, unseen corpus of documents.

In [27]:
other_topics = [i for i in range(tm.n_topics) if i not in tech_topics]
other_texts = [d[0] for d in tm.get_docs(topic_ids=other_topics)]

Let's score these documents and place into a Pandas dataframe.

In [28]:
# score documents based on similarity
other_scores = tm.score(other_texts)

In [29]:
# display results in Pandas dataframe
other_preds = [int(score > 0) for score in other_scores]
data = sorted(list(zip(other_preds, other_scores, other_texts)), key=lambda item:item[1], reverse=True)
print('Top Inliers (or Most Similar to Our Technology-Related Topics')
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head()

Top Inliers (or Most Similar to Our Technology-Related Topics
		Number of Predicted Inliners: 377


,Prediction,Score,Text
0,1,0.212587,I'm looking for recommendations for a laser printer. It will\nbe used mostly for text by a single user. It doesn't need to\nbe a postscript printer. Any advice would be appreciated.\n
1,1,0.211690,"I get the picture, I just find it humorous that Running Windows 3.1 apps ( 3.0 for 2.0 ) \nis what makes os/2 more credible..."
2,1,0.211690,"Two-part question:\n\n1) What is Windows NT - a 'real' windows OS?\n\n2) This past weekend, a local 'hacker' radio show metioned a new product\n from Microsoft called 'Chicago' if I recall. Anyone know what this is?\n\nThat is it -\n\nThanks a heap.\n\n- Alan\n"
3,1,0.211690,Is there any one know:\n\nWhat is the FTP tool for Windows and where to get the tool ?\n\nThanks for any help !!
4,1,0.205488,"Could someone point me toward a source (FTP/BBS/whatever) for development\ntools for the 8051 microprocessor. I specifically am looking for a Macintosh\ncross-assembler/disassembler. Also, is there a mailing-list dedicated to\ndiscussing the 8051? Thanks.\n"


As you can see, we see we've found additional technology-related posts in the dataset.

Our `scorer` assigns a score to each document, where  higher scores indicate a  higher degree of similarity to technology-related seed docments.  The `scorer` implements a decision function to make binary decisions on similarity such that documents with positive scores are deemed as similar and negative scores are deemed dissimilar. We've used this to create  a prediction of 1 for similar and 0 for dissimilar.  This identifies 377 documents as similar.   The `scorer`, however, employs a One-Class classifier, which tends to be more strict.  That is, there are likely documents with negative scores close to zero that are also similar.  Let's look at these.

In [30]:
df[df.Score <=0].head()

,Prediction,Score,Text
377,0,-0.000711,"I was at avalon today and found texture maps in some ""tex"" and ""txc""\nformat, something I've never encountered before. These are obviously\nnot tex or LaTeX files.\n\nIF you have a clue how I can convert these to something\nreasonable, please let me know."
378,0,-0.002478,"I need to be able to cause a beep, but without using any interrupt\nroutines, as I cannot use the BIOS. I believe that the PIC might have\nsomething to do with it, but I'm having troubles deciphering the\ninformation I have on it to figure out how to program it!\n\n\tI'm programming all of this in Turbo C, if that makes any\ndiference at all...\n\n\tPlease can anyone help me??!\n\nThanks,"
379,0,-0.003216,"\nThe only things you'll be able to salvage from the junior are the floppy drives\nand monitor. The floppies are 360k, and the monitor is CGA, but you will need\nan adaptor cable to use it. The junior does not use standard cards. Unless \nyou're really strapped for cash, you should just junk the thing and buy new \nstuff.\n\nDan\n"
380,0,-0.003326,"\nMacintosh II cx with 40 MB HD, 8 MB RAM and 19"" monochrome\nmonitor (Ikegami) is for sale.\nAsking $3,000, no reasonable (best) offer will be rejected.\nContact Konrad at (416) 365-0564m Mon-Frii 9-5.\n"
381,0,-0.003883,"I edited a few newsgroup from that line (don't like to crosspost THAT\nmuch). I can't compare the two, but I recently got an HP DeskJet 500.\n\nI'm very pleased with the output (remember that I'm used to imagens,\nlaser and postscript printers at school -- looks very good. You have\nto be careful to let it dry before touching it, as it will smudge.\n\nThe deskjet is SLOW. This is in comparison to the other printers I\nmentioned. I have no idea how the bubblejet compares.\n\nThe interface between Win3.1 and the printer is just dandy, I've not\nhad any problems with it.\n\nHope that helps some.\n\n--Cindy\n\n--\nCindy Tittle Moore"


As you can see, these documents are also similar and related to technology (albeit slightly different aspects of technology than that of our seed set of documents).  Such negatively-scored documents are useful for identifying so-called informative examples.  Since documents are sorted by score (descending order), we can start at the beginning of the dataframe containing negatively-scored documents and add documents to the positive class until we start seeing negative documents that are **not** related to technology.  These informative negative examples can, then, be added to a negative class for training a traditional binary classsifier.  This process is referred to as [active learning](https://en.wikipedia.org/wiki/Active_learning_(machine_learning)).

For instance, in this example, scores below -0.5 start to become **unrelated** to the themes covered by our technical topics.

In [31]:
df[(df.Score<-0.51)].head()

,Prediction,Score,Text
2217,0,-0.510036,"\nDon't forget Chemical Abstracts Service (which is pretty much the international\nclearinghouse for all chemical information), whose former director (Ronald\nWigington) and head of R&D (Nick Farmer) were openly former NSA employees."
2218,0,-0.510492,"From article <1993Apr21.013846.1374@cx5.com>, by tlc@cx5.com:\n\nAccording to my ColoRIX manual .SCF files are 640x480x256\n\n\nYou may try VPIC, I think it handles the 256 color RIX files OK..\n"
2219,0,-0.510556,What about disks? Won't it erase them if you're carrying them in the bag?
2220,0,-0.510582,"was\nYuppies\nstarted\nYep, that's when I noticed it too. I stopped replacing the hood badge \nafter the second or third one (at $12.00 each).\n\n2002 drivers used to flash their headlight at each other in greeting. Try \nflashing your headlights at a 318i driver and see what kind of look you \nget. They usually check their radar detector...they think you're alerting \nthem to a cop."
2221,0,-0.510692,"refrettably you are mistaken. alt.drugs was used to recruit people for the\nworldwide pot religion. I, however hve no problem being in both of them\n\n"


## Using Keyword Searches to Construct Seed Sets

Let's construct a set of seed documents from a keyword search instead of by LDA-discovered topics.  Let's search all the documents for the word 'Christ':

In [32]:
results = tm.search('Christ', case_sensitive=False)

There are 313 of them.

In [33]:
len(results)

313

Many documents in this set are about Christianity, as expected:

In [34]:
print(results[0][0])


Yep, that's pretty much it. I'm not a Jew but I understand that this is the
Jewish way of thinking. However, the Jews believe that the Covenant between
YHWH and the Patriarchs (Abraham and Moses, in this case) establishes a Moral
Code to follow for mankind. Even the Jews could not decide where the boundaries
fall, though.

As I understand it, the Sadducees believed that the Torah was all that was
required, whereas the Pharisees (the ancestors of modern Judaism) believed that
the Torah was available for interpretation to lead to an understanding of
the required Morality in all its nuances (->Talmud).

The essence of all of this is that Biblical Morality is an interface between
Man and YHWH (for a Jew or Christian) and does not necessarily indicate
anything about YHWH outside of that relationship (although one can speculate).


The trouble with all of this is that we don't really know what the "created
in His image" means. I've heard a number of different opinions on this and
have still

However, since we compiled the seed set of documents based on the keyword "Christ", some documents in the set may be only loosely related to Christianity (if at all). We will see below how this impacts results.

Let's construct a positive class from these 313 documents and use them to find other religious documents:

In [35]:
# compile doc_ids
doc_ids = [doc[1] for doc in results]

# train scorer from document IDs returned by keyword search
tm.train_scorer(doc_ids=doc_ids)

# get text and scores of remaining documents
other_texts = [d[0] for d in tm.get_docs() if d[1] not in doc_ids]
other_scores = tm.score(other_texts)

# display results in Pandas dataframe
other_preds = [int(score > 0) for score in other_scores]
data = sorted(list(zip(other_preds, other_scores, other_texts)), key=lambda item:item[1], reverse=True)
print('Top Inliers (or Most Similar to Our Technology-Related Topics')
print('\t\tNumber of Predicted Inliners: %s' % sum(other_preds))
df = pd.DataFrame(data, columns=['Prediction', 'Score', 'Text'])
df.head(3)

Top Inliers (or Most Similar to Our Technology-Related Topics
		Number of Predicted Inliners: 4759


Prediction     Score  \
0           1  0.418117   
1           1  0.409673   
2           1  0.405350   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Here, we've easily found other documents about religion-focused documents that **do not** explicitly mention Christ. Note that our One-Class classifier predicted a positive label for 4759 documents here. The reason for this large number is that some of the documents in the seed set containing the word 'Christ' may be only loosely related to religion or Christinity (if at all).  

This document, for example, mentions 'Christ' somewhere in the post, but is largely unrelated to Christanity or religion:

In [36]:
print(texts[11311][:1024])



Pardon me? History shows that within the last 170 years, Greeks played 
that game twice: They used Istanbul Patriarch Grigorios in 1822 to 
instigate the Morea rebellion that resulted in the massacres of 
the Muslim people. Again, the Orthodox Patriarch Constantine V 
invited the Russian Czar Nicholas II to invade the Ottoman Empire 
'in the name of Jesus,' and save his flock from Ottoman rule. 

Source: "The 'Past' in Medieval and Modern Greek Culture," in Speros
         Vryonis, ed., 'Byzantina kai Metabyzantina,' Vol I (Malibu,
         Calif., 1978).

p. 161.

In the words of Professor Skiotis, "With savage jubilance, [the Greeks]
sang the words 'Let no Turk remain in the Morea, nor in the whole world.'
The Greeks were determined to achieve to 'Romaiko' in the only way they
knew how: through a war of religious extermination."


  <<The leader of the Ashkenazi community of Corlu complained to the
  president of AIU [Alliance Israelite Universelle] in 1902 about
  persistent Greek

When including such documents in the seed set, documents **unrelated** to Christianity or religion are predicted with a positive label (i.e, false positives).  However, higher scoring documents **will** be related to Christianity, since documents in this dataset containing "Christ" are more likely to be about Christianity and relgion.  Thus, as a document similarity scorer, constructing a positive class by pulling in documents based on keywords can still work.

## Recommending Similar Documents

In the previous section, given a set of seed documents, we scored **new** documents based on similarity.  Here, we will reverse this process. Given a **new** document, we will find (or recommend) documents that are semantically similar to it from the 20newsgroup corpus.

We must first train the recommender.

In [37]:
tm.train_recommender()

Now, let's create some text about space exploration and recommend the top newsgroup posts similar to this text.

In [38]:
rawtext = """
            Elon Musk leads Space Exploration Technologies (SpaceX), where he oversees
            the development and manufacturing of advanced rockets and spacecraft for missions
            to and beyond Earth orbit.
            """

Here is the top recommended 20newsgroup post based on semantic similarity to the text above.

In [39]:
for i, doc in enumerate(tm.recommend(text=rawtext, n=1)):
    print('RESULT #%s'% (i+1))
    print('TEXT:\n\t%s' % (doc[0]))
    print('NEWSGROUP:\n\t%s'% (categories[doc[1]]))
    print('TOPIC:\n\t%s' % (tm.topics[doc[3]]))
    print()

RESULT #1
TEXT:
	Archive-name: space/new_probes
Last-modified: $Date: 93/04/01 14:39:17 $

UPCOMING PLANETARY PROBES - MISSIONS AND SCHEDULES

    Information on upcoming or currently active missions not mentioned below
    would be welcome. Sources: NASA fact sheets, Cassini Mission Design
    team, ISAS/NASDA launch schedules, press kits.


    ASUKA (ASTRO-D) - ISAS (Japan) X-ray astronomy satellite, launched into
    Earth orbit on 2/20/93. Equipped with large-area wide-wavelength (1-20
    Angstrom) X-ray telescope, X-ray CCD cameras, and imaging gas
    scintillation proportional counters.


    CASSINI - Saturn orbiter and Titan atmosphere probe. Cassini is a joint
    NASA/ESA project designed to accomplish an exploration of the Saturnian
    system with its Cassini Saturn Orbiter and Huygens Titan Probe. Cassini
    is scheduled for launch aboard a Titan IV/Centaur in October of 1997.
    After gravity assists of Venus, Earth and Jupiter in a VVEJGA
    trajectory, the spacecr

## Saving and Restoring the Topic Model

The topic model can be saved and restored as follows.

**Save the Topic Model:**

In [40]:
tm.save('/tmp/tm')

**Restore the Topic Model and Rebuild the Document-Topic Matrix**

In [41]:
tm = ktrain.text.load_topic_model('/tmp/tm')

done.


In [42]:
tm.build(texts, threshold=0.25)

done.


Note that the scorer and recommender are not saved, only the LDA topic model is saved.  So, the scorer and recommender should be retrained prior to use as follows:

In [43]:
tm.train_recommender()

In [44]:
rawtext = """
            Elon Musk leads Space Exploration Technologies (SpaceX), where he oversees
            the development and manufacturing of advanced rockets and spacecraft for missions
            to and beyond Earth orbit.
            """

In [45]:
print(tm.recommend(text=rawtext, n=1)[0][0])

Archive-name: space/new_probes
Last-modified: $Date: 93/04/01 14:39:17 $

UPCOMING PLANETARY PROBES - MISSIONS AND SCHEDULES

    Information on upcoming or currently active missions not mentioned below
    would be welcome. Sources: NASA fact sheets, Cassini Mission Design
    team, ISAS/NASDA launch schedules, press kits.


    ASUKA (ASTRO-D) - ISAS (Japan) X-ray astronomy satellite, launched into
    Earth orbit on 2/20/93. Equipped with large-area wide-wavelength (1-20
    Angstrom) X-ray telescope, X-ray CCD cameras, and imaging gas
    scintillation proportional counters.


    CASSINI - Saturn orbiter and Titan atmosphere probe. Cassini is a joint
    NASA/ESA project designed to accomplish an exploration of the Saturnian
    system with its Cassini Saturn Orbiter and Huygens Titan Probe. Cassini
    is scheduled for launch aboard a Titan IV/Centaur in October of 1997.
    After gravity assists of Venus, Earth and Jupiter in a VVEJGA
    trajectory, the spacecraft will arrive a